# Module 8: 10 - Anatomy of an AI Agent - Implementing the Agent Tool Class
---------------------------------------------------------------------------------
In this lesson, we will define the `AgentTool` Python class to integrate the capabilities of converting tool schemas to OpenAI Function Calling format and validating the LLM's tool suggestions. This ensures that the suggested arguments align with the tool function signature before execution. By creating this class, we streamline the tool execution process, combining schema conversion, validation, and execution in a unified structure.

## Objectives
* Define the AgentTool class to manage tool schemas and execution.
* Integrate Pydantic models for schema validation and conversion.
* Validate LLM tool suggestions against the function signature.
* Execute tools with validated arguments using the AI Agent.

## What this session covers:
* Defining the current agent structure, including the LLM Client and short-term memory.
* Creating the AgentTool class to handle tool schemas and execution.
* Converting Python function signatures to OpenAI Function Calling format using Pydantic.
* Validating LLM tool suggestions with Pydantic models.
* Integrating and testing the AgentTool class within the AI Agent framework.

## Install Libraries

In [2]:
#! pip install openai

## Define Current Agent Structure

### LLM Client

In [24]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

### Short-Term Memory

In [25]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""
    
    def __init__(self):
        self.messages = []
    
    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)
    
    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)
    
    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()
    
    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

### Agent Base

In [26]:
from typing import Dict

class Agent:
    """Integrates LLM client, tools, and memory."""
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history, tools=self.tools)
        self.memory.add_message(response)
        return response

## Implementing The Agent Tool Class

In [27]:
from pydantic import BaseModel
from typing import Callable, Type

class AgentTool:
    """Encapsulates a Python function with Pydantic validation."""
    def __init__(self, func: Callable, args_model: Type[BaseModel]):
        self.func = func
        self.args_model = args_model
        self.name = func.__name__
        self.description = func.__doc__ or self.args_schema.get('description', '')

    def to_openai_function_call_definition(self) -> dict:
        """Converts the tool to OpenAI Function Calling format."""
        schema_dict = self.args_schema
        description = schema_dict.pop("description", "")
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": description,
                "parameters": schema_dict
            }
        }

    @property
    def args_schema(self) -> dict:
        """Returns the tool's function argument schema as a dictionary."""
        schema = self.args_model.model_json_schema()
        schema.pop("title", None)
        return schema

## Define Tool Decorator

In [28]:
from typing import Callable, Optional, Type
from pydantic import BaseModel

def check_docstring(func: Callable):
    """Ensure the function has a docstring."""
    if not func.__doc__:
        raise ValueError(f"Function '{func.__name__}' must have a docstring.")

def Tool(func: Optional[Callable] = None, *, args_model: Type[BaseModel]) -> AgentTool:
    """Decorator to wrap a function with an AgentTool instance."""
    def decorator(f: Callable) -> AgentTool:
        check_docstring(f)
        return AgentTool(f, args_model=args_model)
    return decorator(func) if func else decorator

## Define Agent Tools with Tool Decorator

In [29]:
from pydantic import BaseModel, Field
import random

class GetWeatherSchema(BaseModel):
    """Get weather information based on location."""
    location: str = Field(description="Location to get weather for")

@Tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Gets weather information."""
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

In [30]:
from pydantic import BaseModel, Field

class JumpSchema(BaseModel):
    """Jump a specific distance"""
    distance: str = Field(description="Specific distance to jump for")

@Tool(args_model=JumpSchema)
def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

## Convert Python Functions to OpenAI Function Calling Format

### Current Pydantic Approach

In [31]:
from pydantic import BaseModel

def to_openai_function_call_definition(name: str, model: BaseModel):
    schema_dict = model.model_json_schema()
    description = schema_dict.pop("description", "No description provided")
    schema_dict.pop("title", None)  # Remove the title field to exclude the model name
    return {
        "type": "function",
        "function": {
            "name": name,
            "description": description,
            "parameters": schema_dict
        }
    }

get_weather_func_dict = to_openai_function_call_definition("get_weather", GetWeatherSchema)
jump_func_dict = to_openai_function_call_definition("jump", JumpSchema)

tools = [get_weather_func_dict, jump_func_dict]

### New Pydantic Approach

In [32]:
get_weather_func_dict = get_weather.to_openai_function_call_definition()
jump_func_dict = jump.to_openai_function_call_definition()

tools = [get_weather_func_dict, jump_func_dict]

In [33]:
"""
get_weather_func_dict = {
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get weather information based on location.',
        'parameters': {
            'properties': {'location': {'type': 'string'}},
            'required': ['location'],
            'type': 'object'
        }
    }
}
"""
tools[0]

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get weather information based on location.',
  'parameters': {'properties': {'location': {'description': 'Location to get weather for',
     'title': 'Location',
     'type': 'string'}},
   'required': ['location'],
   'type': 'object'}}}

## Improve Agent Tool Class with Tool Argument Validation

In [34]:
from pydantic import BaseModel, ValidationError
from typing import Callable, Type
from inspect import signature

class AgentTool:
    """Encapsulates a Python function with Pydantic validation."""
    def __init__(self, func: Callable, args_model: Type[BaseModel]):
        self.func = func
        self.args_model = args_model
        self.name = func.__name__
        self.description = func.__doc__ or self.args_schema.get('description', '')

    def to_openai_function_call_definition(self) -> dict:
        """Converts the tool to OpenAI Function Calling format."""
        schema_dict = self.args_schema
        description = schema_dict.pop("description", "")
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": description,
                "parameters": schema_dict
            }
        }

    @property
    def args_schema(self) -> dict:
        """Returns the tool's function argument schema as a dictionary."""
        schema = self.args_model.model_json_schema()
        schema.pop("title", None)
        return schema

    def validate_json_args(self, json_string: str) -> bool:
        """Validate JSON string using the Pydantic model."""
        try:
            validated_args = self.args_model.model_validate_json(json_string)
            return isinstance(validated_args, self.args_model)
        except ValidationError:
            return False

    def run(self, *args, **kwargs) -> Any:
        """Execute the function with validated arguments."""
        try:
            # Handle positional arguments by converting them to keyword arguments
            if args:
                sig = signature(self.func)
                arg_names = list(sig.parameters.keys())
                kwargs.update(dict(zip(arg_names, args)))

            # Validate arguments with the provided Pydantic schema
            validated_args = self.args_model(**kwargs)
            return self.func(**validated_args.model_dump())
        except ValidationError as e:
            raise ValueError(f"Argument validation failed for tool '{self.name}': {str(e)}")
        except Exception as e:
            raise ValueError(f"An error occurred during the execution of tool '{self.name}': {str(e)}")

    def __call__(self, *args, **kwargs) -> Any:
        """Allow the AgentTool instance to be called like a regular function."""
        return self.run(*args, **kwargs)

## Re-Define Agent Tools with Tool Decorator

In [35]:
from pydantic import BaseModel, Field
import random

class GetWeatherSchema(BaseModel):
    """Get weather information based on location."""
    location: str = Field(description="Location to get weather for")

@Tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Gets weather information."""
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

In [36]:
from pydantic import BaseModel, Field

class JumpSchema(BaseModel):
    """Jump a specific distance"""
    distance: str = Field(description="Specific distance to jump for")

@Tool(args_model=JumpSchema)
def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

## Validate Agent Tool Class

In [37]:
tool_json_arguments = '{\n  "location": "Virginia"\n}'

# Validate the JSON string arguments
get_weather.validate_json_args(tool_json_arguments)  # True

# Convert JSON string to dictionary and run the tool
import json
tool_arguments_dict = json.loads(tool_json_arguments)
get_weather.run(**tool_arguments_dict)

'Virginia: 66F.'